Data preprocessing

In [ ]:
https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

In [1]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split='train')

C:\Users\sbknl\Documents\GitHub\deeplearning\.data\train.csv: 29.5MB [00:01, 23.9MB/s]                                 


In [2]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')
counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [3]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

In [4]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\sbknl\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [26]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)  # tensor of shape (n_words)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)  # the location where the next sample start, needed since torch.cat(text_list)
    text_list = torch.cat(text_list)  # of shape (the number of all words in the batch)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [ ]:
model receives 338 integers, corresponding to words in a batch
offsets required to cut each batch
with offsets, each batch can have different sizes without padding

In [28]:
l, t, o = next(iter(dataloader))

In [32]:
t.shape

torch.Size([338])

In [10]:
import torch.nn as nn

In [11]:
>>> embedding_sum = nn.EmbeddingBag(10, 3, mode='sum')
>>> # a batch of 2 samples of 4 indices each
>>> input = torch.LongTensor([1,2,4,5,4,3,2,9])
>>> offsets = torch.LongTensor([0,4])
>>> embedding_sum(input, offsets)

tensor([[ 5.8310, -3.0503, -1.9774],
        [ 1.9972, -0.4054, -3.2266]], grad_fn=<EmbeddingBagBackward>)

In [23]:
embedding_sum(torch.LongTensor([1,2,4,5,4,3,2,9,1,2,3,4,5]), torch.LongTensor([0,4, 8]))

tensor([[ 5.8310, -3.0503, -1.9774],
        [ 1.9972, -0.4054, -3.2266],
        [ 5.4507, -2.8298, -3.3885]], grad_fn=<EmbeddingBagBackward>)

In [12]:
embedding = nn.Embedding(10, 3)

In [21]:
embedding(torch.LongTensor([1,2,4,5]))

tensor([[ 0.3175,  0.7299,  0.3146],
        [ 0.3072,  1.1218,  1.0285],
        [ 1.3412,  1.3789,  1.7488],
        [ 1.2973, -1.0044, -1.0377]], grad_fn=<EmbeddingBackward>)

In [19]:
embedding(torch.LongTensor([4,3,2,9]))

tensor([[ 1.3412,  1.3789,  1.7488],
        [-0.7950,  0.0488,  0.2309],
        [ 0.3072,  1.1218,  1.0285],
        [-1.2522,  1.4263, -0.8899]], grad_fn=<EmbeddingBackward>)

In [15]:
embedding(input).sum(axis=0)

tensor([2.8646, 6.2018, 4.1726], grad_fn=<SumBackward1>)

In [7]:
offsets = [0, 100, 200, 300, 400, 500]

In [8]:
torch.tensor(offsets[:-1]).cumsum(dim=0)

tensor([   0,  100,  300,  600, 1000])